In [1]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from scipy import stats

import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np

from sklearn.metrics import accuracy_score, precision_score, recall_score
import random

from sklearn.utils import resample

import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.datasets import imdb
import tensorflow as tf

In [2]:
class UNet(nn.Module):
    def __init__(self, embedding_size=64):
        super(UNet, self).__init__()

        # Define the encoder part
        self.enc_conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.enc_conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)

        # Define the bottleneck part
        self.bottleneck_conv = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(7*7*64, embedding_size)

    def forward(self, x):
        # Encoder
        x = F.relu(self.enc_conv1(x))
        x = self.pool(x)
        x = F.relu(self.enc_conv2(x))
        x = self.pool(x)

        # Bottleneck
        x = F.relu(self.bottleneck_conv(x))
        x = self.flatten(x)
        x = self.fc(x)

        return x

model = UNet()

In [3]:
def imbalance_classes(X, y, imb_type='exp', imb_factor=0.01):
    np.random.seed(42)
    unique_classes = np.unique(y)
    num_classes = len(unique_classes)
    img_max = len(X) / num_classes

    img_num_per_cls = []
    if imb_type == 'exp':
        for cls_idx in range(num_classes):
            num = img_max * (imb_factor**(cls_idx / (num_classes - 1.0)))
            img_num_per_cls.append(int(num))
    elif imb_type == 'step':
        for cls_idx in range(num_classes // 2):
            img_num_per_cls.append(int(img_max))
        for cls_idx in range(num_classes // 2):
            img_num_per_cls.append(int(img_max * imb_factor))
    else:
        img_num_per_cls.extend([int(img_max)] * num_classes)

    X_imb, y_imb = [], []
    for cls, img_num in zip(unique_classes, img_num_per_cls):
        idx = np.random.choice(np.where(y == cls)[0], img_num, replace=False)
        X_imb.append(X[idx])
        y_imb.append(y[idx])

    X_imb = np.concatenate(X_imb)
    y_imb = np.concatenate(y_imb)

    return X_imb, y_imb

In [4]:
def load_imbalanced_cifar10(imbalance_type='exp', imbalance_ratio=0.01):
    (X_train, y_train), (X_test, y_test) = cifar10.load_data()
    X_train_imb, y_train_imb = imbalance_classes(X_train, y_train.flatten(), imbalance_type, imbalance_ratio)
    return X_train_imb, X_test, y_train_imb, y_test.flatten()

In [5]:
X_train_imb, X_test, y_train_imb, y_test = load_imbalanced_cifar10(imbalance_type='exp', imbalance_ratio=0.1)

In [6]:
class UNet(nn.Module):
    def __init__(self, embedding_size=64):
        super(UNet, self).__init__()

        # Define the encoder part
        self.enc_conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.enc_conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)

        # Define the bottleneck part
        self.bottleneck_conv = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(7*7*64, embedding_size)

    def forward(self, x):
        # Encoder
        x = F.relu(self.enc_conv1(x))
        x = self.pool(x)
        x = F.relu(self.enc_conv2(x))
        x = self.pool(x)

        # Bottleneck
        x = F.relu(self.bottleneck_conv(x))
        x = self.flatten(x)
        x = self.fc(x)

        return x

model = UNet()

In [7]:
model(tf.convert_to_tensor(X_train_imb[0]))

TypeError: conv2d() received an invalid combination of arguments - got (tensorflow.python.framework.ops.EagerTensor, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!tensorflow.python.framework.ops.EagerTensor!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, int)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!tensorflow.python.framework.ops.EagerTensor!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, int)
